In [ ]:
!pip install pretrainedmodels

In [ ]:
from PIL import Image
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd

import torch
import fastai
from fastai import vision

import pretrainedmodels as pm

In [ ]:
%matplotlib inline

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Creating the data

To create the training data set we use standard data augmentation techniques. All parameters defining the transformations used for data augmentation are left at their default values (unless otherwise specified).

In [ ]:
_transform_kwargs = {"do_flip": True,
                     "flip_vert": True,  # default is False
                     "max_rotate": 180,  # default is 10
                     "max_zoom": 1.2,    # default is 1.1
                     "max_lighting": 0.2,
                     "max_warp": 0.2,
                     "p_affine": 0.75,
                     "p_lighting": 0.7,
                    }
        
_transforms = vision.get_transforms(**_transform_kwargs)

_data_bunch_kwargs = {"path": "../input/train",
                      "train": "train",
                      "valid_pct": 0.2,
                      "bs": 16,
                      "size": 448,
                      "ds_tfms": _transforms,
                      "test": "../test/test"}

image_data_bunch = (vision.ImageDataBunch
                          .from_folder(**_data_bunch_kwargs)
                          .normalize())

In [ ]:
image_data_bunch.train_ds

In [ ]:
image_data_bunch.valid_ds

In [ ]:
image_data_bunch.test_ds

# Exploring the data

Always important to understand what the images that are being fed into your model actually look like.


In [ ]:
image_data_bunch.show_batch(figsize=(20,20))

# Fitting the model

## Transfer Learning

For computer vision applications always start by trying transfer learning with a standard architecture: [SE-ResNeXt-101](https://arxiv.org/pdf/1803.09820.pdf).

In [ ]:
_base_arch = lambda arg: pm.se_resnext101_32x4d(num_classes=1000, pretrained="imagenet")
learner = vision.cnn_learner(image_data_bunch,
                             base_arch=_base_arch,
                             pretrained=True,
                             metrics=vision.error_rate,
                             model_dir="/kaggle/working/models/se-resnext101-32x4d")

In [ ]:
learner.lr_find()

In [ ]:
(learner.recorder
        .plot())

In [ ]:
def find_optimal_lr(recorder):
    """Extract the optimal learning rate from recorder data."""
    optimal_lr = 0
    minimum_loss = float("inf")
    for loss, lr in zip(recorder.losses, recorder.lrs):
        if loss < minimum_loss:
            optimal_lr = lr
            minimum_loss = loss
    return optimal_lr, minimum_loss


In [ ]:
# define a callback that stores state of "best" model.
# N.B. best model is re-loaded when training completes
_save_model_kwargs = {"every": "improvement",
                      "monitor": "valid_loss",
                      "name": "best-model-stage-1"}
_save_model = (fastai.callbacks
                     .SaveModelCallback(learner, **_save_model_kwargs))

# if validation loss < training loss either learning rate too low or not enough training epoch
learner.fit_one_cycle(15, callbacks=[_save_model])

# Exploring the model's predictions

In [ ]:
clf_interp = (vision.ClassificationInterpretation
                    .from_learner(learner))

In [ ]:
clf_interp.plot_top_losses(16, figsize=(20,20))

In [ ]:
clf_interp.plot_top_losses(16, figsize=(20,20))

In [ ]:
clf_interp.plot_confusion_matrix()

In [ ]:
clf_interp.most_confused()

## Unfreezing, fine-tuning, and learning rates

In [ ]:
learner.unfreeze()

In [ ]:
learner.lr_find()

In [ ]:
(learner.recorder
        .plot())

In [ ]:
_save_model_kwargs = {"every": "improvement",
                      "monitor": "valid_loss",
                      "name": "best-model-stage-2"}
_save_model = (fastai.callbacks
                     .SaveModelCallback(learner, **_save_model_kwargs))
learner.fit_one_cycle(15, max_lr=slice(1e-6, 1e-4), callbacks=[_save_model])

## Test-Time Augmentation (TTA)

In [ ]:
predicted_class_probabilities, _ = learner.TTA(ds_type=fastai.basic_data.DatasetType.Test)

## Creating a submission

In [ ]:
_predicted_classes = (predicted_class_probabilities.argmax(dim=1)
                                                   .numpy())
_class_labels = np.array(['cbb','cbsd','cgm','cmd','healthy'])
_predicted_class_labels = _class_labels[_predicted_classes]

_filenames = np.array([item.name for item in image_data_bunch.test_ds.items])

submission = (pd.DataFrame
                .from_dict({'Category': _predicted_class_labels,'Id': _filenames}))

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', header=True, index=False)